In [ ]:
from hyrax import Hyrax

h = Hyrax()

In [ ]:
h.config["train"]["epochs"] = 1
h.train()

In [ ]:
ds = h.infer()  # Run inference and return the resulting dataset

In [ ]:
# Access patterns for the resulting InferenceDataset
aa = ds[100]  # Retrieve a single index
bb = ds[0:10:2]  # A slice of indices
# cc = ds[[0, 2, 4, 6, 8]]  # Or provide a list of indices to return
dd = ds["asdf"]  # Iterable "index", results in an error message and returns None

In [ ]:
ds.get_id([10])

In [ ]:
h.save_to_database()

In [ ]:
import numpy as np
import json
import pyarrow as pa
import pyarrow.parquet as pq

tensors_0 = [np.random.rand(3, 4, 5) for _ in range(512)]
tensor_shape = tensors_0[0].shape
ids_0 = np.arange(512).astype(str)

tensors_1 = [np.random.rand(3, 4, 5) for _ in range(512)]
ids_1 = np.arange(512).astype(str)

In [ ]:
### Current implementation in Inference dataset

buffer = {}
# _shape = json.dumps(tensors_0[0].shape)

flattened_model_output = [o.flatten() for o in tensors_0]

buffer["id"] = buffer.get("id", []) + list(ids_0)
buffer["model_output"] = buffer.get("model_output", []) + flattened_model_output

table = pa.Table.from_arrays([buffer["id"], buffer["model_output"]], names=["id", "model_output"])

pq.write_to_dataset(table, root_path="./output")

for k in buffer.keys():
    buffer[k] = []

In [ ]:
### New experimental implementation that doesn't require unflattening the numpy arrays

buffer = {}
shape = tensors_0[0].shape

tensors_pa = pa.FixedSizeListArray.from_arrays(np.array(tensors_0).flatten(), np.prod(shape))
ty = pa.fixed_shape_tensor(pa.float64(), shape=shape)
arr = pa.FixedShapeTensorArray.from_storage(ty, tensors_pa)
# table = pa.table({"tensor": arr})

# flattened_model_output = [o.flatten() for o in tensors_0]

buffer["id"] = buffer.get("id", []) + list(ids_0)
buffer["model_output"] = buffer.get("model_output", []) + flattened_model_output

table = pa.Table.from_arrays([buffer["id"], arr], names=["id", "model_output"])

pq.write_to_dataset(table, root_path="./output")

for k in buffer.keys():
    buffer[k] = []

In [ ]:
data = pq.read_table(
    "/home/drew/code/hyrax/docs/pre_executed/output/7fd0474d170d4d9fb5518653f414f8d8-0.parquet"
)

data["model_output"][0]

In [ ]:
buffer = {}
shape = tensors_0[0].shape

# tensors_pa = pa.FixedSizeListArray.from_arrays(np.array(tensors_0).flatten(), np.prod(shape))
# ty = pa.fixed_shape_tensor(pa.float64(), shape=shape)
arr = pa.FixedShapeTensorArray.from_numpy_ndarray(np.array(tensors_0))
arr2 = pa.FixedShapeTensorArray.from_numpy_ndarray(np.array(tensors_1))
# table = pa.table({"tensor": arr})

# flattened_model_output = [o.flatten() for o in tensors_0]

buffer["id"] = buffer.get("id", []) + list(ids_0)

table = pa.Table.from_arrays([buffer["id"], arr], names=["id", "model_output"])

pq.write_to_dataset(table, root_path="./output")

for k in buffer.keys():
    buffer[k] = []

In [ ]:
data = pq.read_table(
    "/home/drew/code/hyrax/docs/pre_executed/output/acc93f5f5b594cee8281b81c97352b89-0.parquet"
)
test = data["model_output"][3]

for i in test:
    print(i.to_numpy().shape)
    print(i.to_numpy().ndim)

In [ ]:
np.array([str(i) for i in data["id"][0]])

In [ ]:
pddata = data.to_pandas()
pddata["model_output"].iloc[0]